In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer("nvidia/NV-Embed-v2",trust_remote_code=True)
print(model.max_seq_length)

C:\Users\valla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [02:20<00:00, 35.10s/it]


32768


In [35]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import pickle
import os

with open('Chunks/chunks_400_100.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

chunks=loaded_data

In [36]:
corpus =[chunk.page_content for chunk in chunks]
corpus_embeddings = model.encode(corpus, normalize_embeddings=True)



In [41]:
query = "What materials are acceptable for fireblocking in residential construction?"
query_embedding = model.encode(query, normalize_embeddings=True)

dot_scores = util.dot_score(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(dot_scores, k=5)



In [42]:
Returned_retrieved_chunks=[]
for score, idx in zip(top_results[0], top_results[1]):
    Returned_retrieved_chunks.append(corpus[idx])

Returned_retrieved_chunks

['DEFINITIONS\n2018 INTERNATIONAL RESIDENTIAL CODE ® 17\n[RB] FIREBLOCKING.  Building materials or materials\napproved for use as fireblocking, installed to resist the free\npassage of flame to other areas of the building through con-\ncealed spaces.\n[RB] FIREPLACE. An assembly consisting of a hearth and\nfire chamber of noncombustible material and provided with a\nchimney, for use with solid fuels.',
 'Fireblocking  . . . . . . . . . . . . . . . . . . . . . . . . . R302.11\nFloors . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . R302.13\nFoam plastics . . . . . . . . . . . . . . . . . . . . . . . . . R302.8\nInsulation  . . . . . . . . . . . . . . . . . . . R302.10, R302.14\nPenetrations . . . . . . . . . . . . . . . . . . . . . . . . . . R302.4',
 'R302.11.1 Fireblocking materials. Except as provided in\nSection R302.11, Item 4, fireblocking shall consist of the\nfollowing materials.\n1. Two-inch (51 mm) nominal lumber.\n2. Two thicknesses of 1-inch (25.4 mm) nominal lum-

In [47]:
Returned_retrieved_chunks=[]
for score, idx in zip(top_results[0], top_results[1]):
    Returned_retrieved_chunks.append(corpus[idx])


import requests
import json

url = 'http://localhost:11434/api/generate'
payload = {
    "model": "llama3",
    "prompt": f"I will provide you text please do you use that text only to give answer for my query. Text:{Returned_retrieved_chunks}, query:{query}, "
}
data = json.dumps(payload)

response = requests.post(url, data=data, headers={'Content-Type': 'application/json'})

if response.status_code == 200:
    list_dict_words = []
    for each_word in response.text.split("\n"):
        try:
            data = json.loads(each_word) 
        except:
            pass
        list_dict_words.append(data)
        


llama_response = " ".join([word['response'] for word in list_dict_words if type(word) == type({})])
print(llama_response)

Based  on  the  provided  text ,  according  to  Section  R 302 . 11 . 1 ,  the  following  materials  are  acceptable  for  fire blocking  in  residential  construction :

 1 .  Two -inch  ( 51  mm )  nominal  lumber .
 2 .  Two  thickness es  of   1 -inch  ( 25 . 4  mm )  nominal  lumber  with  broken  lap  joints .
 3 .  One  thickness  of   23 / 32 -inch  ( 18 . 3  mm )  wood  structural  panels  with  joints  backed  by   23 / 32 -inch  ( 18 . 3  mm )  wood  structural  panels .
 4 .  Other  approved  non rig id  materials ,  such  as  unf aced  fiberglass  batt  insulation ,  used  for  horizontal  fire blocking  in  walls  constructed  using  parallel  rows  of  studs  or  stagger ed  studs .

 Please  note  that  these  materials  are  accepted  for  compliance  with  the   10 -foot  ( 304 8  mm )  horizontal  fire blocking  in  walls  constructed  using  parallel  rows  of  studs  or  stagger ed  studs .  
